In [1]:
import pandas as pd
import itertools
import numpy as np
import openpyxl
# from scenario_generation import generate_fuel_prices


### SETS

In [2]:
fuel_price_csv = 'fuel_price2_1.csv'
# Velg antall av de ulike settene her
set_sheets = {'Power systems': 11, 'Ages': 6, 'Routes': 3122, 'Time periods':6, 'Time periods 1': 2, 'Time periods 2':4, 'Fuel types': 10, 'Installations': 18, 'Scenarios': 100}

t_values = range(0, 6)  # time period from 1 to 6
t1_values = range(0, 2)
t2_values = range(2, 6)
f_values = range(1, 10)  # Assuming you want values from 1 to 7 for 'f'
r_values = range(1, 1819)  # Assuming you want values from 1 to 3122 for 'r'
n_values = range(1, 19) # 18 installations
s_values = range(1,12) # 11 systems
w_values = range(1, 11) # 100 scenarios
a_values = range(0, 6)

for t in f_values:
    print(t)


1
2
3
4
5
6
7
8
9


In [3]:
# Skriver alle sets med verdier definert over til excel

sets_excel_output_path = '../run_model/SetData-sheets.xlsx'

set_sheets = {
    'Power systems': s_values,
    'Ages': a_values,
    'Routes': r_values,
    'Time periods': t_values,
    'Time periods 1': t1_values,
    'Time periods 2': t2_values,
    'Fuel types': f_values,
    'Installations': n_values,
    'Scenarios': w_values
}

# Create data frames for each set and write to Excel
with pd.ExcelWriter(sets_excel_output_path, engine='openpyxl') as writer:
    for sheet, values in set_sheets.items():
        # Create a DataFrame for the current set
        df = pd.DataFrame(itertools.product(values), columns=[sheet])

        # Write the DataFrame to the Excel file with the current sheet name
        df.to_excel(writer, sheet_name=sheet, index=False)

#### Generere kombinasjoner av indekser til parameterne

In [4]:

combinations_f_r = itertools.product(f_values, r_values)
combinations_ems = itertools.product(f_values, r_values)
combinations_r_s = itertools.product(r_values, s_values)
combinations_t = itertools.product(t_values)
combinations_s_a = itertools.product(s_values, a_values)
combinations_f_r_w = itertools.product(f_values, r_values, w_values)
combinations_w = itertools.product(w_values)

#### Inputdata

In [5]:
# Hardkodet inn fuelpris per MJ per fueltype 1-6
fuel_prices = {1: 0.192, 2:0.173, 3:0.367, 4:0.328, 5:0.237, 6:0.222, 7: 0.214, 8: 0.202, 9: 0.307}




fuel_dict = {
    1: (0.18, 0.09, 'fossil'),      # MGO
    2: (0.15, 0.07, 'fossil'),      # LNG
    3: (0.47, 0.26, 'green_fuels'), # GREEN METHANOL
    4: (0.41, 0.22, 'green_fuels'), # GREEN NH3
    5: (0.28, 0.15, 'bio'),         # B30
    6: (0.24, 0.13, 'bio'),         # B20
    7: (0.23, 0.12, 'bio'),         # B15
    8: (0.20, 0.11, 'bio'),         # B70
    9: (0.43, 0.18, 'green_fuels')  # Helelektrisk
}
 # Example fuel bounds with group information


# Karbonutslipp per MJ som er brukt for hver type fuel
emissions_carbon_tonn_per_MJ_wtw = {1: 0.000182, 2: 0.000153, 3: 0.000033, 4: 0.000033, 5: 0.000134, 6: 0.000150, 7: 0.000158, 8: 0.000170, 9: 0.000008 }
emissions_carbon_tonn_per_MJ_ttw = {1: 0.000152778, 2: 0.000115556, 3: 0.000085278, 4: 0.000025583, 5: 0.000017528, 6: 0.0, 7: 0.0}
emissions_carbon_tonn_per_MJ = emissions_carbon_tonn_per_MJ_wtw 


# Regne ut reduksjonskrav for karbonutslipp
init_ems_tonn = 165000

scenario_1_reductions = {0: 0, 1: 0.50, 2: 0.55, 3: 0.70, 4: 0.80, 5: 0.95 } # ambisjonene til norske myndigheter. t=1 = 2030
scenario_2_reductions = {0: 0, 1: 0, 2: 0.29, 3: 0.49, 4: 0.65, 5: 0.78, 6: 0.87} # DNV report Norway 2023 - kutt kun innenfor transport
scenario_0_reductions = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0}

# Velg hvilket scenario som skal kjøres
reduction_requirements = scenario_1_reductions

capacity_systems = {1: 100, 2: 100, 3: 100, 4: 100, 5: 70, 6: 70, 7: 70, 8: 70, 9: 90, 10: 90, 11: 70, 12: 90, 13: 90}

#### Lese inn data fra rutegenereringsprogrammet

In [6]:
# # Leser inn nødvendig MJ på å kjøre hver rute
mj_route = pd.read_csv('../route_generation/generated_datafiles/mj_route.csv')
mj_route_el = pd.read_csv('../route_generation/generated_datafiles/mj_route_strøm.csv')



# # Leser inn tid på å kjøre hver rute
hours_route = pd.read_csv('../route_generation/generated_datafiles/hours_route.csv')
hours_route_el = pd.read_csv('../route_generation/generated_datafiles/hours_route_strøm.csv')

demand_el = pd.read_csv('../route_generation/generated_datafiles/demand_strøm.csv')
demand = pd.read_csv('../route_generation/generated_datafiles/demand.csv')

# Skriver verdiene tilbake i excelfilen for alle parametrene
output_excel_path = '../run_model/Parameterdata-sheets.xlsx'

##### Generere P_w

In [7]:
df_probability = pd.DataFrame(combinations_w, columns=['O'])

df_probability['Value'] = 1/len(list(w_values))

df_probability.head()


,O,Value
0,1,0.1
1,2,0.1
2,3,0.1
3,4,0.1
4,5,0.1


In [8]:

with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_probability.to_excel(writer, sheet_name='Probability', index=False)

##### Distance dict for compatibility og cost of fuel

In [9]:
# Lager dictionaries for MJ_route for å senere kunne multiplisere sammen disse verdiene for å få kostnad per rute per fueltype
distance_dict = mj_route['mj_route'].to_dict()
#distance_dict = distance_dict['mj_route']

# lager egne distanser for elektrisk

specific_distance_dict = mj_route_el['mj_route'].to_dict()
#specific_distance_dict= specific_distance_dict['mj_route']
electric_distance_dict = {key + 1: value for key, value in specific_distance_dict.items()}

#print(distance_dict)
distance_dict = {key + 1: value for key, value in distance_dict.items()}
print(len(distance_dict))
mj_route

print(len(electric_distance_dict))

1818
1818


##### Compatibility RS

In [10]:
# Lager dictionaries for MJ_route for å senere kunne multiplisere sammen disse verdiene for å få kostnad per rute per fueltype
demand_dict = demand['Demand'].to_dict()
#demand_dict = demand_dict['Demand']

# lager egne distanser for elektrisk

specific_demand_dict = demand_el['Demand'].to_dict()
#specific_demand_dict= specific_demand_dict['Demand']

electric_demand_dict = {key + 1: value for key, value in specific_demand_dict.items()}

demand_dict = {key + 1: value for key, value in demand_dict.items()}
print(demand_dict)
demand

print(electric_demand_dict)

{1: 85.25, 2: 92.25, 3: 71.75, 4: 76.0, 5: 86.75, 6: 96.25, 7: 75.75, 8: 96.25, 9: 75.75, 10: 81.25, 11: 81.25, 12: 95.92, 13: 85.25, 14: 95.75, 15: 85.25, 16: 96.25, 17: 90.75, 18: 94.25, 19: 78.25, 20: 75.42, 21: 82.75, 22: 75.25, 23: 75.75, 24: 70.25, 25: 73.75, 26: 83.0, 27: 72.0, 28: 72.0, 29: 71.67, 30: 79.0, 31: 71.5, 32: 89.75, 33: 72.0, 34: 87.0, 35: 87.0, 36: 72.0, 37: 72.0, 38: 86.67, 39: 76.0, 40: 94.0, 41: 86.5, 42: 76.0, 43: 87.0, 44: 81.5, 45: 85.0, 46: 76.0, 47: 75.67, 48: 83.0, 49: 75.5, 50: 93.75, 51: 76.0, 52: 70.5, 53: 74.0, 54: 75.67, 55: 83.0, 56: 75.5, 57: 93.75, 58: 76.0, 59: 70.5, 60: 74.0, 61: 99.25, 62: 99.25, 63: 96.25, 64: 78.75, 65: 78.75, 66: 82.67, 67: 75.17, 68: 93.42, 69: 75.67, 70: 70.17, 71: 73.67, 72: 72.0, 73: 82.75, 74: 82.5, 75: 72.0, 76: 83.0, 77: 77.5, 78: 81.0, 79: 93.25, 80: 75.5, 81: 73.5, 82: 82.75, 83: 93.75, 84: 88.25, 85: 91.75, 86: 75.75, 87: 70.5, 88: 74.0, 89: 94.0, 90: 94.0, 91: 79.0, 92: 79.0, 93: 93.67, 94: 83.0, 95: 93.5, 96: 83.0

In [11]:
# Lage combinations for rs
# For hver rad med r s, sjekke om kapasiteten i s er større enn eller lik demanden til r. Sett til 1 hvis true, 0 ellers

# Create a DataFrame from the combinations
df_routecap = pd.DataFrame(combinations_r_s, columns=['R', 'S'])

# Display the first few rows of the DataFrame
print(df_routecap.head())

df_routecap['Value'] = 0

# Iterate over the DataFrame
for index, row in df_routecap.iterrows():
    r = row['R']
    s = row['S']
    if s != 11 and demand_dict.get(r, 0) <= capacity_systems.get(s, float('inf')):
        df_routecap.at[index, 'Value'] = 1
    if s == 11 and electric_demand_dict.get(r, 0) <= capacity_systems.get(s, float('inf')):
        df_routecap.at[index, 'Value'] = 1

print(df_routecap)

   R  S
0  1  1
1  1  2
2  1  3
3  1  4
4  1  5
          R   S  Value
0         1   1      1
1         1   2      1
2         1   3      1
3         1   4      1
4         1   5      0
...     ...  ..    ...
19993  1818   7      0
19994  1818   8      0
19995  1818   9      0
19996  1818  10      0
19997  1818  11      0

[19998 rows x 3 columns]


In [12]:
with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_routecap.to_excel(writer, sheet_name='Compatibility rs', index=False)

##### Cost of fuel per route

In [13]:

# Lager dictionaries for MJ_route for å senere kunne multiplisere sammen disse verdiene for å få kostnad per rute per fueltype
distance_dict = mj_route['mj_route'].to_dict()
#distance_dict = distance_dict['mj_route']

# lager egne distanser for elektrisk

specific_distance_dict = mj_route_el['mj_route'].to_dict()
#specific_distance_dict= specific_distance_dict['mj_route']
electric_distance_dict = {key + 1: value for key, value in specific_distance_dict.items()}

#print(distance_dict)
distance_dict = {key + 1: value for key, value in distance_dict.items()}
print(len(distance_dict))
mj_route

print(len(electric_distance_dict))

1818
1818


Generere alle kombinasjoner av R og F

In [14]:
# Create a DataFrame from the combinations
df_fuelprice = pd.DataFrame(combinations_f_r, columns=['F', 'R'])

# Display the first few rows of the DataFrame
print(df_fuelprice.head())




   F  R
0  1  1
1  1  2
2  1  3
3  1  4
4  1  5


#### Fuelcost

In [15]:

# regner ut verdi for hver kombinasjon av fueltype og rute gitt verdiene i dictionariene over
def calculate_value(row):
    distance = electric_distance_dict[row['R']] if row['F'] == 6 else distance_dict[row['R']]
    return fuel_prices[row['F']] * distance

# Legger til verdiene i Value-kolonne i dataframen
df_fuelprice['Value'] = df_fuelprice.apply(calculate_value, axis=1)

# df_fuelprice

##### Fuelcost 2

In [16]:
fuel_price2_df = pd.read_csv(fuel_price_csv)

fuel_price2_df.head()

,F,R,T,S,Value
0,1,1,2,1,26096.703695
1,1,2,2,1,22867.805308
2,1,3,2,1,23478.116864
3,1,4,2,1,21063.563532
4,1,5,2,1,23818.580700


##### Skriver Cost of fuel til excel

In [17]:
# Using ExcelWriter to write to the existing Excel file
with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    fuel_price2_df.to_excel(writer, sheet_name='Cost of fuel 2', index=False)

# Using ExcelWriter to write to the existing Excel file
with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_fuelprice.to_excel(writer, sheet_name='Cost of fuel 1', index=False)

##### EMISSIONS FOR FUEL TYPE

Generere alle kombinasjoner av F og R i kolonnene


In [18]:
# Create a DataFrame from the combinations
df_ems = pd.DataFrame(combinations_ems, columns=['F', 'R'])

# Display the first few rows of the DataFrame
print(df_ems.head())

   F  R
0  1  1
1  1  2
2  1  3
3  1  4
4  1  5


In [19]:
# multipliserer carbon per MJ per fueltype med distanse seilt per rute. 
# def calculate_emissions(row):
#     distance = electric_distance_dict[row['R']] if row['F'] == 6 else distance_dict[row['R']]
#     return emissions_carbon_tonn_per_MJ[row['F']] * distance
print(electric_distance_dict)
def calculate_emissions(row):
    distance = electric_distance_dict[row['R']] if row['F'] == 6 else distance_dict[row['R']]
    return emissions_carbon_tonn_per_MJ[row['F']] * distance



# Apply the function to create a new column 'Value'
df_ems['Value'] = df_ems.apply(calculate_emissions, axis=1)


# Skriver dataframen med emissions til parameter_data excel filen
with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_ems.to_excel(writer, sheet_name='Emissions', index=False)


{1: 290374.4, 2: 250619.6, 3: 259405.2, 4: 229849.6, 5: 262538.0, 6: 268309.2, 7: 271483.2, 8: 273349.2, 9: 280763.2, 10: 233736.4, 11: 288136.4, 12: 242251.2, 13: 258294.4, 14: 257908.0, 15: 278934.4, 16: 259429.2, 17: 246227.6, 18: 239264.4, 19: 245649.2, 20: 260076.8, 21: 286608.4, 22: 276042.0, 23: 259643.2, 24: 265881.6, 25: 259558.4, 26: 291534.8, 27: 314240.0, 28: 322000.0, 29: 282902.0, 30: 307125.2, 31: 297438.8, 32: 309025.2, 33: 302560.0, 34: 375052.8, 35: 345292.8, 36: 306480.0, 37: 362880.0, 38: 321086.4, 39: 316729.6, 40: 328029.6, 41: 311771.6, 42: 337369.6, 43: 348572.8, 44: 282891.2, 45: 270259.6, 46: 323609.6, 47: 325711.6, 48: 378014.8, 49: 341288.4, 50: 335823.2, 51: 321049.6, 52: 328236.4, 53: 320164.8, 54: 339631.6, 55: 391534.8, 56: 353208.4, 57: 378383.2, 58: 339449.6, 59: 337356.4, 60: 327844.8, 61: 293756.4, 62: 372156.4, 63: 305669.2, 64: 315010.4, 65: 314770.4, 66: 295796.8, 67: 283550.4, 68: 315216.8, 69: 306751.6, 70: 273470.0, 71: 268026.8, 72: 228320.0, 

##### MAX EMISSIONS PER TIME PERIOD

In [20]:


# Create a DataFrame from the combinations
df_max_ems = pd.DataFrame(combinations_t, columns=['T'])

# Display the first few rows of the DataFrame
# print(df_max_ems)

In [21]:


# Regner ut max emissions gitt uslippsreduksjonsscenario. Endre scenario-funksjon etter hvilket scenario vi vil kjøre med
def calculate_max_emissions(row):
    return (1 - reduction_requirements[row['T']]) * init_ems_tonn

# Legger til verdiene i Value-kolonne i dataframen
df_max_ems['Value'] = df_max_ems.apply(calculate_max_emissions, axis=1)
# print(df_max_ems.head())
# Skriver dataframen med emissions til parameter_data excel filen
with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_max_ems.to_excel(writer, sheet_name='Max Emissions', index=False)

df_max_ems

,T,Value
0,0,165000.0
1,1,82500.0
2,2,74250.0
3,3,49500.0
4,4,33000.0
5,5,8250.0


##### TIME USED

In [22]:
# Create T_rs
combinations_r_s = itertools.product(r_values, s_values)
df_timeused = pd.DataFrame(combinations_r_s, columns=['R', 'S'])

# Lager dictionaries for MJ_route for å senere kunne multiplisere sammen disse verdiene for å få kostnad per rute per fueltype
route_dict = hours_route['Hours'].to_dict()
hours_route_dict = {key + 1: value for key, value in route_dict.items()}
print(hours_route_dict)
# lager egne distanser for elektrisk
hours_route_el_dict = hours_route_el['Hours'].to_dict()
hours_route_el_dict = {key + 1: value for key, value in hours_route_el_dict.items()}
print(hours_route_el_dict)


def calculate_timeused(row):
    value = hours_route_el_dict[row['R']] if row['S'] == 11 else hours_route_dict[row['R']]
    return value


df_timeused['Value'] = df_timeused.apply(calculate_timeused, axis=1)

print(df_timeused.head())
# Skriver dataframe med emissions til parameter_data excel filen
with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_timeused.to_excel(writer, sheet_name='Time used', index=False)


{1: 48.0, 2: 48.0, 3: 48.0, 4: 48.0, 5: 48.0, 6: 48.0, 7: 48.0, 8: 48.0, 9: 48.0, 10: 48.0, 11: 48.0, 12: 48.0, 13: 48.0, 14: 48.0, 15: 48.0, 16: 48.0, 17: 48.0, 18: 48.0, 19: 48.0, 20: 48.0, 21: 48.0, 22: 48.0, 23: 48.0, 24: 48.0, 25: 48.0, 26: 48.0, 27: 48.0, 28: 48.0, 29: 48.0, 30: 48.0, 31: 48.0, 32: 48.0, 33: 48.0, 34: 48.0, 35: 48.0, 36: 48.0, 37: 48.0, 38: 48.0, 39: 48.0, 40: 48.0, 41: 48.0, 42: 48.0, 43: 48.0, 44: 48.0, 45: 48.0, 46: 48.0, 47: 48.0, 48: 48.0, 49: 48.0, 50: 48.0, 51: 48.0, 52: 48.0, 53: 48.0, 54: 48.0, 55: 48.0, 56: 48.0, 57: 48.0, 58: 48.0, 59: 48.0, 60: 48.0, 61: 48.0, 62: 48.0, 63: 48.0, 64: 48.0, 65: 48.0, 66: 48.0, 67: 48.0, 68: 48.0, 69: 48.0, 70: 48.0, 71: 48.0, 72: 48.0, 73: 48.0, 74: 48.0, 75: 48.0, 76: 48.0, 77: 48.0, 78: 48.0, 79: 48.0, 80: 48.0, 81: 48.0, 82: 48.0, 83: 48.0, 84: 48.0, 85: 48.0, 86: 48.0, 87: 48.0, 88: 48.0, 89: 48.0, 90: 48.0, 91: 48.0, 92: 48.0, 93: 48.0, 94: 48.0, 95: 48.0, 96: 48.0, 97: 48.0, 98: 48.0, 99: 48.0, 100: 48.0, 101: 48

KeyboardInterrupt: 